<a href="https://colab.research.google.com/github/Majids-Hamm/Pharmacovigillance/blob/main/cioms_narration_producer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving cioms interim.rar to cioms interim.rar


In [ ]:
!pip install python-docx

In [ ]:
!pip install rarfile
!apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import rarfile

# Get the name of the first (and presumably only) uploaded .rar file
rar_filename = list(uploaded.keys())[0]
print(f"Unrarring file: {rar_filename}")

with rarfile.RarFile(rar_filename, 'r') as rar_ref:
    rar_ref.extractall('cioms_reports') # Extracts to a folder named 'cioms_reports'

print("Extraction complete. Files extracted to 'cioms_reports' folder.")

NameError: name 'uploaded' is not defined

In [ ]:
!pip install hazm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 17.1 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9300 sha256=6ea548bd635f0b92fb5c303051ada3576bcae8504f5e87c0f82a5a0072cb4a62
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext
  Attempting uninstall: numpy
   

In [ ]:
from hazm import Normalizer

normalizer = Normalizer()

#This function is not currently implemented in the code, since the results were not appealing.
#A single line of code using the function normalizer in the extract cioms data function will do the job if wanted

In [ ]:
import os
from docx import Document
import re
from google.colab import files

# --- Unicode BiDi & Formatting Constants ---
RTL_MARK = '\u200F'
LTR_MARK = '\u200E'
ZWNJ = '\u200C'
PDF = '\u202C'

In [ ]:
def parse_filename_for_info(filename):
    name_without_ext = os.path.splitext(filename)[0]
    match = re.search(r'([A-Z0-9]+PEM-\d{3})-(\d{8})', name_without_ext)
    if match:
        patient_code = match.group(1)
        filename_date_raw = match.group(2)
        return patient_code, filename_date_raw
    return None, None

In [ ]:
def extract_cioms_data(docx_path):
    filename = os.path.basename(docx_path)
    full_patient_code_from_filename, filename_report_date = parse_filename_for_info(filename)

    try:
        doc = Document(docx_path)
    except Exception as e:
        print(f"Error reading document {docx_path}: {e}")
        if full_patient_code_from_filename:
            return full_patient_code_from_filename, filename_report_date or f"FILE_READ_ERROR_DATE_{filename}", f"Error reading document: {e}", None, None
        return None, None, "Error reading document", None, None

    if not doc.tables:
        print(f"Warning: No tables found in {docx_path}")
        error_msg = "No tables found in document"
        if full_patient_code_from_filename:
            return full_patient_code_from_filename, filename_report_date or f"NO_TABLES_DATE_{filename}", error_msg, None, None
        return f"NO_TABLE_{filename}", filename_report_date or f"NO_TABLES_DATE_{filename}", error_msg, None, None

    tables_to_check = doc.tables[:min(3, len(doc.tables))]
    full_text_for_search = ""
    narrative_text_candidate = ""
    sponsor_date_from_doc = None
    gender = None
    age = None

    for table in tables_to_check:
        row_count = len(table.rows)
        for r_idx, row in enumerate(table.rows):
            for c_idx, cell in enumerate(row.cells):
                cell_text = cell.text.strip()
                if not cell_text:
                    continue
                full_text_for_search += "\n" + cell_text

                # --- Extract Sponsor Date ---
                if not sponsor_date_from_doc:
                    sponsor_date_match = re.search(r"Date\s*Received\s*by\s*Sponsor\s*(\d{1,2}\s*/\s*\d{1,2}\s*/\s*\d{4})", cell_text)
                    if sponsor_date_match:
                        sponsor_date_from_doc = sponsor_date_match.group(1).replace(" ", "")

                # --- Extract Gender ---
                if not gender:
                    if "مذکر" in cell_text:
                        gender = "Male"
                    elif "مونث" in cell_text:
                        gender = "Female"

                # --- Extract Age ---
                if not age and re.search(r'\bAge\b', cell_text, re.IGNORECASE):
                    # Try to extract age from same cell
                    age_match = re.search(r'\bAge\b.*?(\d{1,3})', cell_text)
                    if age_match:
                        age = age_match.group(1)
                    else:
                        # Try the cell directly beneath this one
                        if r_idx + 1 < row_count:
                            below_cell_text = table.rows[r_idx + 1].cells[c_idx].text.strip()
                            below_age_match = re.search(r'\d{1,3}', below_cell_text)
                            if below_age_match:
                                age = below_age_match.group(0)

                # --- Extract Narrative Candidate ---
                if not narrative_text_candidate and "بیمار در تاریخ" in cell_text and len(cell_text) > 500:
                    narrative_text_candidate = cell_text

    # --- Fallback Sponsor Date ---
    if not sponsor_date_from_doc:
        sponsor_date_match_fallback = re.search(r"Date\s*Received\s*by\s*Sponsor\s*(\d{1,2}\s*/\s*\d{1,2}\s*/\s*\d{4})", full_text_for_search)
        if sponsor_date_match_fallback:
            sponsor_date_from_doc = sponsor_date_match_fallback.group(1).replace(" ", "")
        else:
            sponsor_date_from_doc = f"UNKNOWN_SPONSOR_DATE_{filename}"

    # --- Final Narrative Extraction ---
    narrative_text = "Narrative text could not be located."
    SUMMARY_MARKER = "جمع‌بندی:"

    def protect_persian(text):
        text = text.replace('\u200C', ZWNJ)
        text = text.replace(RTL_MARK, '').replace(LTR_MARK, '').replace(PDF, '')
        return text

    if narrative_text_candidate:
        narrative_start_index = narrative_text_candidate.find("بیمار در تاریخ")
        if narrative_start_index != -1:
            narrative_body = narrative_text_candidate[narrative_start_index:]
            narrative_body = narrative_body.replace('\u200C', ZWNJ)
            summary_start_index = narrative_body.find(SUMMARY_MARKER)
            if summary_start_index != -1:
                lines_from_summary = narrative_body[summary_start_index:].splitlines()
                end_index = summary_start_index + len(lines_from_summary[0]) if lines_from_summary else summary_start_index + len(SUMMARY_MARKER) + 100
                if len(lines_from_summary) > 1 and "Comment by" not in lines_from_summary[1]:
                    end_index += 1 + len(lines_from_summary[1])
                narrative_text = narrative_body[:end_index].strip()
            else:
                narrative_text = narrative_body[:2000].strip()
        else:
            narrative_text = narrative_text_candidate.strip()
    else:
        full_narrative_start = full_text_for_search.find("بیمار در تاریخ")
        if full_narrative_start != -1:
            full_summary_start = full_text_for_search.find(SUMMARY_MARKER, full_narrative_start)
            if full_summary_start != -1:
                narrative_text = full_text_for_search[full_narrative_start:full_summary_start + len(SUMMARY_MARKER) + 200].strip()
            else:
                narrative_text = full_text_for_search[full_narrative_start:full_narrative_start + 2000].strip()

    narrative_text = protect_persian(narrative_text)

    # --- Extract patient code from content if missing from filename ---
    final_patient_code = full_patient_code_from_filename
    if not final_patient_code:
        for table in tables_to_check:
            for row in table.rows:
                for cell in row.cells:
                    cell_text = cell.text.strip()
                    if not final_patient_code:
                        code_match = re.search(r"که\s+با\s+کد\s+([A-Z0-9]+(?:[A-Z0-9\-]*[A-Z0-9])?)", cell_text)
                        if code_match:
                            raw_code = code_match.group(1).strip(" .,،)([]{}؛")
                            final_patient_code = re.sub(r'\s+', '', raw_code)
                            break
                if final_patient_code:
                    break
            if final_patient_code:
                break
        if not final_patient_code:
            code_match_fallback = re.search(r"که\s+با\s+کد\s+([A-Z0-9]+(?:[A-Z0-9\-]*[A-Z0-9])?)", full_text_for_search)
            if code_match_fallback:
                raw_code = code_match_fallback.group(1).strip(" .,،)([]{}؛")
                final_patient_code = re.sub(r'\s+', '', raw_code)
            else:
                final_patient_code = f"UNKNOWN_CODE_{filename}"

    final_report_date = sponsor_date_from_doc
    if not final_report_date or "UNKNOWN" in final_report_date:
        final_report_date = filename_report_date if filename_report_date else f"UNKNOWN_REPORT_DATE_{filename}"

    return final_patient_code, final_report_date, narrative_text, age, gender


In [ ]:
def extract_patient_number(full_patient_code):
    if not full_patient_code or not isinstance(full_patient_code, str):
        return 9999
    match = re.search(r'.*-([0-9]+)$', full_patient_code)
    if match:
        try:
            return int(match.group(1))
        except ValueError:
            pass
    return 9999


In [ ]:
def process_all_cioms_files(folder_path="./"):
    narratives_list = []
    docx_files = []

    # --- Collect .docx files ---
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.docx'):
                docx_files.append(os.path.join(root, file))

    if not docx_files:
        print("No .docx files found.")
        return narratives_list

    print(f"Found {len(docx_files)} .docx file(s).")

    for file_path in docx_files:
        filename = os.path.basename(file_path)
        print(f"\n--- Processing: {filename} ---")
        try:
            full_patient_code, report_date, narrative_text, age, gender = extract_cioms_data(file_path)
            if full_patient_code and narrative_text and "Error reading document" not in narrative_text:
                patient_number = extract_patient_number(full_patient_code)
                narratives_list.append((patient_number, report_date, full_patient_code, gender, age, narrative_text))
                print(f"  -> Extracted: {full_patient_code} (Num: {patient_number})")
            else:
                print(f"  -> Failed: {filename}")
        except Exception as e:
            print(f"  -> Error: {e}")

    return narratives_list


In [ ]:
# --- Main Execution ---
folder_path = "./cioms_reports"
print(f"Starting processing of files in: {folder_path}")
narratives_list_unsorted = process_all_cioms_files(folder_path)

try:
    sorted_narratives_list = sorted(narratives_list_unsorted, key=lambda x: (x[0], x[1]))
    print(f"\nSuccessfully sorted {len(sorted_narratives_list)} narratives.")
except Exception as e:
    print(f"Error sorting: {e}")
    sorted_narratives_list = narratives_list_unsorted

if sorted_narratives_list:
    combined_output = "\uFEFFCIOMS Narratives\n" + "="*50 + "\n\n"
    for i, (num, date, code, gender, age, narrative) in enumerate(sorted_narratives_list, 1):
        combined_output += f"--- Entry #{i} ---\n"
        combined_output += f"Patient Code: {code} (Num: {num})\n"
        combined_output += f"Report Date: {date}\n"
        combined_output += f"Gender: {gender or 'UNKNOWN'}\n"
        combined_output += f"Age: {age or 'UNKNOWN'}\n"
        combined_output += f"Narrative:\n{narrative}\n"
        combined_output += "---\n\n"

    output_file = "combined_cioms_narratives.txt"
    with open(output_file, 'w', encoding='utf-8-sig') as f:
        f.write(combined_output)
    files.download(output_file)
    print(f"\nOutput saved and downloaded.")
else:
    print("No data extracted.")

Starting processing of files in: ./cioms_reports
Found 295 .docx file(s).

--- Processing: CIOMS Pembrolizumab-MAZA117PEM-007-14020802.docx ---
  -> Extracted: MAZA117PEM-007 (Num: 7)

--- Processing: CIOMS Pembrolizumab-MARA123PEM-010 -14021207.docx ---
  -> Extracted: MARA123PEM-010 (Num: 10)

--- Processing: CIOMS Pembrolizumab-MOEB128PEM-121-14031127.docx ---
  -> Extracted: MOEB128PEM-121 (Num: 121)

--- Processing: CIOMS Pembrolizumab-ABMO123PEM-130-140300716.docx ---
  -> Extracted: ABMO123PEM-130 (Num: 130)

--- Processing: CIOMS Pembrolizumab-MARA123PEM-257-14040214-1.docx ---
  -> Extracted: MARA123PEM-257 (Num: 257)

--- Processing: CIOMS Pembrolizumab-LENA138PEM-012-14030202.docx ---
  -> Extracted: LENA138PEM-012 (Num: 12)

--- Processing: CIOMS Pembrolizumab-HARE145PEM-076-14030621.docx ---
  -> Extracted: HARE145PEM-076 (Num: 76)

--- Processing: CIOMS Pembrolizumab-RONO326PEM-022-14030523.docx ---
  -> Extracted: RONO326PEM-022 (Num: 22)

--- Processing: CIOMS Pembroliz

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Output saved and downloaded.
